In [7]:
# Chatgpt Code for splitting mp4 video to jpg for AlphaPose halpe fine tuning
import cv2

read_path = "E:\\SkiProject\\Original_videos\\DJI_0044.MP4"
save_path = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\"
vidcap = cv2.VideoCapture(read_path)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(f"{save_path}frame_{count:06d}.jpg", image)     # Save frame as JPEG file
    success, image = vidcap.read()
    count += 1


In [ ]:
# chatgpt code for replacing png woth jpg in CVAT annotation file
import json

# Load the JSON file
file_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Update file names in the "images" section to change .PNG to .jpg
for image in data.get("images", []):
    image["file_name"] = image["file_name"].replace(".PNG", ".jpg")

# Save the updated JSON file
updated_file_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated.json'
with open(updated_file_path, 'w') as f:
    json.dump(data, f, indent=4)



'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated.json'

In [8]:
import os
import json
import random
import shutil

# File paths (reloading after reset)
json_file_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated.json'
images_folder = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\"  # Assuming images are in this folder
train_folder = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\train\\"
val_folder = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\val\\"

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Create train and validation folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Shuffle and split the images (80% train, 20% validation)
images = data["images"]
random.shuffle(images)
split_index = int(0.8 * len(images))
train_images = images[:split_index]
val_images = images[split_index:]

# Helper function to move images
def move_images(image_list, dest_folder):
    for image in image_list:
        image_name = image['file_name']
        src_path = os.path.join(images_folder, image_name)
        dest_path = os.path.join(dest_folder, image_name)
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)

# Move train and validation images to their respective folders
move_images(train_images, train_folder)
move_images(val_images, val_folder)

# Filter annotations for train and validation
train_image_ids = {img["id"] for img in train_images}
val_image_ids = {img["id"] for img in val_images}

train_annotations = [ann for ann in data["annotations"] if ann["image_id"] in train_image_ids]
val_annotations = [ann for ann in data["annotations"] if ann["image_id"] in val_image_ids]

# Create new train and validation JSON files
train_data = {
    "info": data["info"],
    "licenses": data["licenses"],
    "categories": data["categories"],
    "images": train_images,
    "annotations": train_annotations
}

val_data = {
    "info": data["info"],
    "licenses": data["licenses"],
    "categories": data["categories"],
    "images": val_images,
    "annotations": val_annotations
}

# Save the train and validation JSON files
train_json_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_train.json'
val_json_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_val.json'

with open(train_json_path, 'w') as f:
    json.dump(train_data, f, indent=4)

with open(val_json_path, 'w') as f:
    json.dump(val_data, f, indent=4)

train_json_path, val_json_path


('E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_train.json',
 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_val.json')